# IFSP - Câmpus Campinas

## Análise sobre a previsibilidade das decisões de admissibilidade no Superior Tribunal de Justiça usando inteligência artificial

#### Aluno: Guilherme Cioldin Dainese

### Parte 1 - Carregamento dos dados abertos do Superior Tribunal de Justiça

O Superior Tribunal de Justiça (STJ) utiliza um portal para disponibilizar dados do Tribunal para acesso público. O portal está disponível no endereço https://dadosabertos.web.stj.jus.br/.

O dataset específico para esse trabalho é o "Íntegras de Decisões Terminativas e Acórdãos do Diário da Justiça", com o link específico: https://dadosabertos.web.stj.jus.br/dataset/integras-de-decisoes-terminativas-e-acordaos-do-diario-da-justica.

Esse dataset possui as decisões monocráticas indicadas pelos gabinetes de ministros como terminativas e que tenham sido proferidas diretamente no processo principal.

O dataset possui três tipos de arquivos. Primeiro, um arquivo .csv que é utilizado unicamente como dicionário das entradas nos arquivos JSON. Os arquivos JSON por sua vez possuem os metadados das decisões e por fim, cada entrada possui um arquivo .zip com o texto das decisões em si.

Os arquivos .zip serão ignorados pois os dados serão processados diretamente das petições constantes dos processos judiciais.

Todavia, será feito o download dos arquivos .json para construir a base de dados inicial que permitirá a seleção das petições.

In [1]:
from pprint import pprint
from time import sleep
from tqdm.notebook import tqdm
import json
import pandas as pd
import re
import requests
import os
from time import sleep

In [2]:
#Endereço da raiz da base de dados do CNJ
api_url_root = 'https://dadosabertos.web.stj.jus.br/'
#Lista de pacotes seguindo o esquema da API CKAN
package_list = 'api/3/action/package_list'

In [3]:
#Exibe a lista de pacotes
pprint(requests.get(api_url_root + package_list).json())

{'help': 'https://dadosabertos.web.stj.jus.br/api/3/action/help_show?name=package_list',
 'result': ['api-publica-datajud',
            'atas-de-distribuicao',
            'espelhos-de-acordaos-corte-especial',
            'espelhos-de-acordaos-primeira-secao',
            'espelhos-de-acordaos-primeira-turma',
            'espelhos-de-acordaos-quarta-turma',
            'espelhos-de-acordaos-quinta-turma',
            'espelhos-de-acordaos-segunda-secao',
            'espelhos-de-acordaos-segunda-turma',
            'espelhos-de-acordaos-sexta-turma',
            'espelhos-de-acordaos-terceira-secao',
            'espelhos-de-acordaos-terceira-turma',
            'integras-de-decisoes-terminativas-e-acordaos-do-diario-da-justica',
            'movimentacao-processual',
            'plano-estrategico-stj',
            'precedentes-qualificados'],
 'success': True}


O pacote está identificado como `integras-de-decisoes-terminativas-e-acordaos-do-diario-da-justica`

In [4]:
package_id = 'integras-de-decisoes-terminativas-e-acordaos-do-diario-da-justica'

In [5]:
#Carrega dados de pacote da API do STJ
response = requests.get(f'{api_url_root}/api/3/action/package_show?id={package_id}') #Comando padrão da API CKAN para carregar os dados
package_info = json.loads(response.text) #Carrega o arquivo .json
resources = package_info['result']['resources'] #Salva os recursos da query

In [6]:
#Exibe o primeiro resultado
resources[0]

{'cache_last_updated': None,
 'cache_url': None,
 'created': '2022-05-19T20:01:02.643048',
 'datastore_active': False,
 'description': 'Dicionário de dados. ',
 'format': 'CSV',
 'hash': '',
 'id': '0488e8aa-33a3-494d-86ef-17889a0155ee',
 'last_modified': '2022-05-19T20:01:02.627455',
 'metadata_modified': '2022-05-19T20:02:05.698389',
 'mimetype': 'text/csv',
 'mimetype_inner': None,
 'name': 'dicionariointegrasdecisoes.csv',
 'package_id': 'a2cd85cc-1391-4ebc-aeed-a45dd75a7987',
 'position': 0,
 'resource_type': None,
 'size': 1536,
 'state': 'active',
 'url': 'https://dadosabertos.web.stj.jus.br/dataset/a2cd85cc-1391-4ebc-aeed-a45dd75a7987/resource/0488e8aa-33a3-494d-86ef-17889a0155ee/download/dicionariointegrasdecisoes.csv',
 'url_type': 'upload'}

Esse resultado é o arquivo .csv que é o dicionário das decisões do STJ nesse dataset.

In [7]:
dicionario = requests.get(resources[0]['url'])
if not os.path.exists('misc'):
    os.makedirs('misc')
if not os.path.exists('misc/dicionariointegrasdecisoes.csv'):
    with open('misc/dicionariointegrasdecisoes.csv', 'w') as file:
        file.write("")
if dicionario.status_code == 200:
        with open(f"./misc/{resources[0]['name']}", 'wb') as f:
            f.write(dicionario.content.decode('iso-8859-1').encode('utf-8'))
#Foi necessário converter a codificação do arquivo.

Salvo o arquivo de dicionário, resta fazer o download dos arquivos de metadados.

Considerando a entrada em vigor da emenda constitucional n.º 125 de 2022, serão selecionados apenas os processos com data de decisão no STJ posterior a setembro de 2022.

In [8]:
# Filtrar apenas os arquivos do tipo JSON
arquivos_json = [resource for resource in resources if resource["format"].lower() == "json"]

Os arquivos variam de nome entre metadados\[ano]\[mês]\[dia].json.

Para filtrar as decisões posteriores a essa data utilizaremos expressões regulares.

In [9]:
#Cria expressão regular para excluir as entradas anteriores a setembro de 2022.
#Não há necessidade de controlar os anos na expressão regular pois a primeira entrada é em março de 2022.
pattern = re.compile(r'metadados20220[1-8]')
json_selecionados = [resource for resource in arquivos_json if not pattern.match(resource["name"])]

In [10]:
#Essa célula é para carregar a lista de documentos utilizada para o projeto é a mesma do original.
#Aqui mantida somente para assegurar a reprodutibilidade dos dados.
salvar_dados = None
if json_selecionados[-1]['name']!='metadados20240115.json':
    with open('misc/lista_documentos.json', 'r') as file:
        json_selecionados = json.load(file)
        salvar_dados = False
        print('Carregada lista de dados original.')
else:
    print('Carregada lista de dados baixadas do portal do STJ.')
    salvar_dados = True

Carregada lista de dados baixadas do portal do STJ.


In [11]:
if salvar_dados == True:
    with open('misc/lista_documentos.json', 'w') as file:
            json.dump(json_selecionados, file, indent=4)
    print('Dados salvos em lista_documentos.json.')

Dados salvos em lista_documentos.json.


In [12]:
#Cria pasta para os metadados em json
if not os.path.exists('json_metadados'):
    os.makedirs('json_metadados')

In [13]:
# Itera sobre todas as entradas selecionadas para baixar os arquivos de metadados
arquivos_falha = []

for resource in tqdm(json_selecionados):
    if os.path.exists(f'json_metadados/{resource['name']}'):
        continue #Pula o download se o arquivo já foi baixado anteriormente
    
    response = requests.get(resource["url"])
    if response.status_code == 200:
        with open('./json_metadados/' + resource["name"], 'xb') as file:
            file.write(response.content)
    else:
        print(f"Falha ao baixar {resource['name']}, status code: {response.status_code}")
        arquivos_falha.append(resource)

  0%|          | 0/314 [00:00<?, ?it/s]

In [14]:
#Execute essa célula quanto necessário até o download de todos os arquivos completar.
#Repita até aparecer que não há mais arquivos a serem baixados.
lista_temp_arquivos = arquivos_falha.copy()
if len(lista_temp_arquivos) == 0:
        print('Não há mais arquivos a serem baixados.')
else:
    for item in lista_temp_arquivos:
        resource = arquivos_falha.pop(0)
        if os.path.exists(f'json_metadados/{resource['name']}'):
            continue #Pula o download se o arquivo já foi baixado anteriormente
        response = requests.get(resource["url"])
        if response.status_code == 200:
            with open('./json_metadados/' + resource["name"], 'xb') as file:
                file.write(response.content)
            print(f' Arquivo {resource["name"]} baixado com sucesso.')     
        else:
            print(f"Falha ao baixar {resource['name']}, status code: {response.status_code}")
            arquivos_falha.append(resource)

Não há mais arquivos a serem baixados.


Realizado o download, resta apenas unir os arquivos em um único dataframe para posterior processamento.

In [15]:
#Une todos os metadados em um único dataframe
dfs = []
for entrada_json in json_selecionados:
    df = pd.read_json(f"./json_metadados/{entrada_json['name']}")
    dfs.append(df)
#Retorna um dataframe único com todos os dados baixados
dataframe = pd.concat(dfs, ignore_index=True)

In [16]:
#Exibe do dataframe concatenado
dataframe

,seqDocumento,dataPublicacao,tipoDocumento,numeroRegistro,processo,dataRecebimento,dataDistribuicao,ministro,recurso,teor,descricaoMonocratica,assuntos
0,163410022,1662001200000,ACÓRDÃO,202103427345,REsp 2005877,1634871600000,1635303600000,NANCY ANDRIGHI,None,Concedendo,None,5825;5825
1,162915181,1662001200000,ACÓRDÃO,202102019306,REsp 1946637,1624849200000,1626318000000,MANOEL ERHARDT (DESEMBARGADOR CONVOCADO DO TRF5),AGRAVO INTERNO,Concedendo,None,6060;6060;5994
2,163409979,1662001200000,ACÓRDÃO,202102897496,REsp 1959435,1630638000000,1632366000000,NANCY ANDRIGHI,None,Concedendo,None,10470;10470
3,163239479,1662001200000,ACÓRDÃO,201301198451,REsp 1382933,1366945200000,1371006000000,ASSUSETE MAGALHÃES,AGRAVO INTERNO,Negando,None,10305;10221
4,163227292,1662001200000,ACÓRDÃO,202201220906,REsp 1999207,1651201200000,1653015600000,MARCO BUZZI,AGRAVO INTERNO,Negando,None,10496;10496;14919;14920
...,...,...,...,...,...,...,...,...,...,...,...,...
751909,224027345,1705284000000,DECISÃO,202301309450,AREsp 2343988,1681959600000,1684465200000,REYNALDO SOARES DA FONSECA,None,Concedendo,Agravo conhecido para dar parcial provimento a...,10621;3637;5555;5566;10621;12334
751910,223999858,1705284000000,DECISÃO,202301132414,HC 814406,1680577200000,1681095600000,REYNALDO SOARES DA FONSECA,None,Concedendo,Não conhecido o Habeas Corpus. Concedido o Hab...,3628;3628;3628
751911,224007770,1705284000000,DECISÃO,202301118658,AREsp 2330257,1680577200000,1681354800000,REYNALDO SOARES DA FONSECA,AGRAVO REGIMENTAL,Concedendo,Agravo conhecido para dar provimento ao Recurs...,3372;3372;10637
751912,224040816,1705284000000,DECISÃO,202002489460,AREsp 1764530,1600743600000,1601953200000,SEBASTIÃO REIS JÚNIOR,None,Concedendo,Agravo conhecido para dar provimento ao Recurs...,3608;3608


Todos os dados relevantes disponíveis no repositório já foram baixados e unidos em um único dataframe. Salvaremos o arquivo para seguir com o processamento.

A opção de arquivo (feather) é utilizada para permitir maior rapidez de salvamento e menor espaço.

In [17]:
dataframe.shape
#751914 rows × 12 columns

(751914, 12)

In [18]:
#Cria pasta para os metadados em json
if not os.path.exists('dataframes'):
    os.makedirs('dataframes')
dataframe.to_feather('./dataframes/json_raw.feather')